In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from binance.spot import Spot
import seaborn as sns
import requests

# Close all figures before switching the backend
plt.close('all')
matplotlib.use('TkAgg')  # Use the TkAgg backend for interactive plotting

In [2]:
# Function to get historical data from Upbit
def get_upbit_data(market, to, count):
    url = f"https://api.upbit.com/v1/candles/days"
    params = {
        'market': market,
        'to': to,
        'count': count
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

# Define the market and date range for Upbit
market = "USDT-BTC"
end_date = "2023-01-02T00:00:00Z"  # Upbit uses ISO 8601 format
count = 200  # Number of days to retrieve per request

# Retrieve data in batches
all_upbit_data = []
while True:
    data = get_upbit_data(market, end_date, count)
    if not data:
        break
    all_upbit_data.extend(data)
    end_date = data[-1]['candle_date_time_utc']  # Update the end date to the last retrieved candle's date

# Organize the data into a DataFrame
upbit_columns = ['candle_date_time_utc', 'opening_price', 'high_price', 'low_price', 'trade_price', 'candle_date_time_kst']
df_upbit = pd.DataFrame(all_upbit_data, columns=upbit_columns)

# Convert necessary columns to the correct data type
df_upbit['candle_date_time_utc'] = pd.to_datetime(df_upbit['candle_date_time_utc'])
df_upbit['candle_date_time_kst'] = pd.to_datetime(df_upbit['candle_date_time_kst'])
df_upbit['trade_price'] = df_upbit['trade_price'].astype(float)  # Ensure 'trade_price' is a float for calculations

# Rename columns to match Binance DataFrame
df_upbit.rename(columns={
    'candle_date_time_utc': 'Open Time',
    'opening_price': 'Open',
    'high_price': 'High',
    'low_price': 'Low',
    'trade_price': 'Close',
    'candle_date_time_kst': 'Close Time'
}, inplace=True)

# Select only the required columns
df_upbit = df_upbit[['Open Time', 'Open', 'High', 'Low', 'Close', 'Close Time']]


# Sort the DataFrame by 'Open Time' from 2021/01/01 to 2023/01/01
df_upbit_sorted = df_upbit[(df_upbit['Open Time'] >= '2021-01-01') & (df_upbit['Open Time'] <= '2023-01-01')].sort_values(by='Open Time')

In [3]:
df_upbit_sorted

,Open Time,Open,High,Low,Close,Close Time
730,2021-01-01,29079.509886,29470.133500,27854.055000,29292.510000,2021-01-01 09:00:00
729,2021-01-02,28895.423000,34000.000000,28809.475000,32061.954640,2021-01-02 09:00:00
728,2021-01-03,32126.078549,35945.000000,31766.595000,33333.545100,2021-01-03 09:00:00
727,2021-01-04,33139.006175,34922.097600,28313.934444,31893.612640,2021-01-04 09:00:00
726,2021-01-05,31779.001000,35000.000000,28020.000000,34066.266000,2021-01-05 09:00:00
...,...,...,...,...,...,...
4,2022-12-28,16613.511008,16904.283392,16510.025401,16510.173204,2022-12-28 09:00:00
3,2022-12-29,16510.404608,16880.922903,16500.000000,16551.690572,2022-12-29 09:00:00
2,2022-12-30,16551.690572,16876.919829,16321.887787,16748.679992,2022-12-30 09:00:00
1,2022-12-31,16538.402582,16927.379800,16340.880402,16791.928097,2022-12-31 09:00:00


In [4]:
df_upbit_sorted['Pct Change'] = df_upbit_sorted['Close'].pct_change()
df_upbit_sorted['Pct Change'].dropna()

729    0.094544
728    0.039660
727   -0.043198
726    0.068122
725    0.084475
         ...   
4     -0.008696
3      0.002515
2      0.011901
1      0.002582
0     -0.014748
Name: Pct Change, Length: 730, dtype: float64

In [6]:
# Initialize the Binance client
client = Spot()

In [5]:
# Step 1: Define start and end dates for data retrieval
start_date = "2021-01-01"
end_date = "2023-01-01"  # Adjusted end date for forward testing

#seconds to milliseconds
start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)

In [7]:
# Step 2: Retrieve the historical daily data for BTC/USDT pair
# Initialize an empty list to store all the klines
all_klines = []

# Retrieve data in batches
while start_timestamp < end_timestamp:
    klines = client.klines("BTCUSDT", "1d", startTime=start_timestamp, endTime=end_timestamp, limit=1000)
    if not klines:
        break
    all_klines.extend(klines)
    start_timestamp = klines[-1][0] + 1  # Update the start timestamp to the last retrieved kline's open time

In [8]:
# Step 3: Organize the data into a DataFrame
columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
df_binance = pd.DataFrame(all_klines, columns=columns)
# Select only the required columns
df_binance = df_binance[['Open Time', 'Open', 'High', 'Low', 'Close', 'Close Time']]

In [9]:
# Step 4: Convert necessary columns to the correct data type
df_binance['Open Time'] = pd.to_datetime(df_binance['Open Time'], unit='ms')
df_binance['Close Time'] = pd.to_datetime(df_binance['Close Time'], unit='ms')
df_binance['Close'] = df_binance['Close'].astype(float)  # Ensure 'Close' is a float for calculations


In [10]:
df_binance

,Open Time,Open,High,Low,Close,Close Time
0,2021-01-01,28923.63000000,29600.00000000,28624.57000000,29331.69,2021-01-01 23:59:59.999
1,2021-01-02,29331.70000000,33300.00000000,28946.53000000,32178.33,2021-01-02 23:59:59.999
2,2021-01-03,32176.45000000,34778.11000000,31962.99000000,33000.05,2021-01-03 23:59:59.999
3,2021-01-04,33000.05000000,33600.00000000,28130.00000000,31988.71,2021-01-04 23:59:59.999
4,2021-01-05,31989.75000000,34360.00000000,29900.00000000,33949.53,2021-01-05 23:59:59.999
...,...,...,...,...,...,...
726,2022-12-28,16706.06000000,16785.19000000,16465.33000000,16547.31,2022-12-28 23:59:59.999
727,2022-12-29,16547.32000000,16664.41000000,16488.91000000,16633.47,2022-12-29 23:59:59.999
728,2022-12-30,16633.47000000,16677.35000000,16333.00000000,16607.48,2022-12-30 23:59:59.999
729,2022-12-31,16607.48000000,16644.09000000,16470.00000000,16542.40,2022-12-31 23:59:59.999


In [11]:
# Step 5: Calculate percentage change in 'Close' prices
df_binance['Pct Change'] = df_binance['Close'].pct_change()
df_binance['Pct Change'].dropna()

1      0.097050
2      0.025536
3     -0.030647
4      0.061297
5      0.083059
         ...   
726   -0.009520
727    0.005207
728   -0.001563
729   -0.003919
730    0.004495
Name: Pct Change, Length: 730, dtype: float64

In [12]:
# Check if 'Open Time' column exists in both DataFrames
if 'Open Time' in df_binance.columns and 'Open Time' in df_upbit_sorted.columns:
    # Convert 'Open Time' column to datetime format
    df_binance['Open Time'] = pd.to_datetime(df_binance['Open Time'], unit='ms')
    df_upbit_sorted['Open Time'] = pd.to_datetime(df_upbit_sorted['Open Time'], format='%Y-%m-%d')

    # Set 'Open Time' column as index
    df_binance.set_index('Open Time', inplace=True)
    df_upbit_sorted.set_index('Open Time', inplace=True)

    # Ensure both DataFrames have the same date range
    common_dates = df_binance.index.intersection(df_upbit_sorted.index)
    df_binance = df_binance.loc[common_dates]
    df_upbit_sorted = df_upbit_sorted.loc[common_dates]

In [13]:
# Plot the data with actual dates in the format '%Y%m'
plt.figure(figsize=(10, 6))
df_binance['Close'].plot(label='Binance')
df_upbit_sorted['Close'].plot(label='Upbit')
plt.title('BTC/USDT Close Prices')
plt.xlabel('Date')
plt.ylabel('Close Price (USDT)')
plt.grid(True)
plt.legend()
plt.show()

In [14]:
# Step 6: Split the data into backtesting (50%) and forward testing (50%)
split_index = len(df_binance) // 2
df_backtest = df_binance.iloc[:split_index].copy()
df_forwardtest = df_binance.iloc[split_index:].copy()

In [15]:
# Step 7: Implement the Kimchi Momentum Strategy
def apply_strategy(df_binance, df_upbit_sorted, x_threshold, y_threshold):
    df_binance['Signal'] = 0
    current_position = 0  # 0: no position, 1: long, -1: short

    for i in range(1, len(df_binance)):
        if i >= len(df_upbit_sorted):
            break
        pct_change = df_upbit_sorted['Pct Change'].iloc[i]

        if pct_change > x_threshold and current_position != 1:
            df_binance.at[i, 'Signal'] = 1  # Long position
            current_position = 1
        elif pct_change < -y_threshold and current_position != -1:
            df_binance.at[i, 'Signal'] = -1  # Short position
            current_position = -1
        else:
            df_binance.at[i, 'Signal'] = current_position  # No change in position

    

    # Calculate returns (assuming returns are based on closing price)
    df_binance['Market Return'] = df_binance['Close'].pct_change(fill_method=None).dropna() 
    df_binance['Strategy Return'] = df_binance['Signal'].shift(1) * df_binance['Market Return'] # Shift to reflect prior" day's signal return return df_binance
    return df_binance

In [18]:
# Step 8: Calculate performance metrics
def calculate_metrics(df_binance):
    # CAGR Calculation
    total_return = (df_binance['Strategy Return'].dropna() + 1).prod()
    n_days = df_binance['Strategy Return'].dropna().shape[0]
    cagr = (total_return ** (365 / n_days)) - 1 
    
    # Maximum Drawdown Calculation
    df_binance['Cumulative Return'] = (df_binance['Strategy Return'] + 1).cumprod()
    df_binance['Previous Peaks'] = df_binance['Cumulative Return'].cummax()
    drawdown = ((df_binance['Cumulative Return']) - df_binance['Previous Peaks']) / df_binance['Previous Peaks']
    max_drawdown = drawdown.min()
    
    # Sharpe Ratio Calculation (assuming risk-free rate is 4% for simplicity)
    riskfree_rate = 0.04
    annualized_vol = max(df_binance['Strategy Return'].dropna().std() * np.sqrt(365), 1e-9)
    excess_return = cagr - riskfree_rate
    sharpe_ratio = excess_return / annualized_vol
    
    return cagr, max_drawdown, sharpe_ratio

In [21]:
# Step 9: Optimize X and Y thresholds using backtest data
x_values = np.arange(0.029, 0.061, 0.001)
y_values = np.arange(0.039, 0.061, 0.001)
sharpe_ratios = np.zeros((len(x_values), len(y_values)))

best_sharpe_ratio = -np.inf  # Fix 1
best_x = None
best_y = None

for i, x in enumerate(x_values):
    for j, y in enumerate(y_values):
        df_strategy_backtest = apply_strategy(df_backtest.copy(), df_upbit_sorted, x, y)
        _, _, sharpe_ratio_backtest = calculate_metrics(df_strategy_backtest)
        
        # Fix 2: Skip invalid Sharpe ratios
        if np.isfinite(sharpe_ratio_backtest) and (sharpe_ratio_backtest > best_sharpe_ratio):
            best_sharpe_ratio = sharpe_ratio_backtest
            best_x = x
            best_y = y

# Fix 3: Fallback values
best_x = best_x if best_x is not None else 0.0
best_y = best_y if best_y is not None else 0.0

print(f'Best X: {best_x:.3f}, Best Y: {best_y:.3f}, Best Sharpe Ratio: {best_sharpe_ratio:.3f}')

Best X: 0.029, Best Y: 0.039, Best Sharpe Ratio: -40000000.000


In [ ]:
data = pd.DataFrame(sharpe_ratios, index=y_values, columns=x_values)

# Step 10: Plotting the Sharpe Ratio heatmap with figures in each color block for backtest only
plt.figure(figsize=(32, 26)) 
heatmap = sns.heatmap(data, annot=True, cmap='viridis', cbar_kws={'label': 'Sharpe Ratio'}, fmt=".2f", annot_kws={"size": 9, "color": "white"})

plt.xlabel('X Threshold (%)')
plt.ylabel('Y Threshold (%)')
plt.title('Sharpe Ratio Heatmap for Kimchi Momentum Strategy (Backtest Only)')

plt.savefig('sharpe_ratio_heatmap_backtest_only.png')  # Save the plot as an image file

# Display the heatmap
from PIL import Image
sharpe_heatmap_img = Image.open('sharpe_ratio_heatmap_backtest_only.png')
sharpe_heatmap_img.show()

In [ ]:
# Step 10: Apply the optimized X and Y to forward test data
df_strategy_forwardtest = apply_strategy(df_forwardtest, best_x, best_y)
cagr_forwardtest, max_drawdown_forwardtest, sharpe_ratio_forwardtest = calculate_metrics(df_strategy_forwardtest)

print(f'Forward Test Results - CAGR: {cagr_forwardtest:.2f}, Max Drawdown: {max_drawdown_forwardtest:.2f}, Sharpe Ratio: {sharpe_ratio_forwardtest:.2f}')